In [5]:
import rasterio
import geopandas as gpd
from rasterio.features import rasterize
from rasterio.windows import from_bounds
import numpy as np

In [6]:
# random_sample_path="../../sample_collection/palisades_random_sample.shp"
# sar_image_path="../MachineLearning/input/palisades_s1_ready.tif"
# output_gt_image_path="output/gt/palisades_gt.tif"

random_sample_path="../../sample_collection/eaton_random_sample.shp"
reference_image_path="../MachineLearning/input/eaton_s1_ready.tif"
output_gt_image_path="output/gt/eaton_gt.tif"

In [ ]:
gdf = gpd.read_file(random_sample_path)

with rasterio.open(reference_image_path) as dataset:
    band_1=dataset.read(1)

    meta = dataset.meta.copy()
    meta.update({
            'count': 1,
        })
    
    count=dataset.count

    print(gdf.itertuples())
    gdf = gdf.rename(columns={"class": "class_value"})

    # Extract geometries and class values
    shapes = [(geom, row.class_value) for geom, row in zip(gdf.geometry, gdf.itertuples())]

    # Rasterize shapefile with the given class values
    rasterized = rasterize(
        shapes=shapes,
        out_shape=band_1.shape,
        transform=meta['transform'],
        fill=2,  # Default value if no features exist
        dtype=band_1.dtype
    )

    # bands = dataset.read()
    
    # Write the clipped data to a new file
    with rasterio.open(output_gt_image_path, 'w', **meta) as dest:
        dest.write(rasterized, 1)  # Write the first band



### Rearrange bands


In [ ]:
## Eaton
mul_sar_image_path="../../SAR_Data_processing_eaton/8_subset/mul_stack/eaton_S1A_IW_GRDH_1SDV_TC_mul_stack_32611_clip_bilinear_10.tif"
# 16_dec [1,2], 2_feb[3,4], 28_dec[5,6], 21_jan[7,8]
# [1,2][5,6][7,8][3,4]

minx, miny, maxx, maxy=390284.9643946937, 3778493.0553008374, 407654.9643946937, 3791083.0553008374
clip_extent = (minx, miny, maxx, maxy)

In [7]:
##palisades
sar_image_path_1="../../SAR_Data_processing/8_1_project/S1A_IW_GRDH_1SDV_palisades_32611_bilinear_10.tif"
# 28_dec_vh, vv, 14_jan_vh,vv 
sar_image_path_2="../../SAR_Data_processing/8_0_subset/exported_from_eaton/palisades_S1A_IW_GRDH_1SDV_2024_12_16_2024_01_21_TC_stack_32611_bilinear_10.tif"
# 16_dec_vh, vv, 2_feb, 21jan_vh, vv
minx, miny, maxx, maxy=330142.6891275004,3757093.028339643,375622.6891275004,3794643.028339643
clip_extent = (minx, miny, maxx, maxy)


In [9]:
output_image_path="output/sar/1_palisades_s1_mul_ready.tif"
with rasterio.open(sar_image_path_1) as dataset:
    

    meta=dataset.meta.copy()
    window = from_bounds(*clip_extent, dataset.transform)
    
    # Get the new transform for the clipped data
    clipped_transform = dataset.window_transform(window)
    clipped_data = dataset.read(1, window=window)
    # Define the metadata for the clipped raster
    meta.update({
        'height': clipped_data.shape[0],
        'width': clipped_data.shape[1],
        'transform': clipped_transform
    })
    vh_pre_2=dataset.read(1, window=window)
    vv_pre_2=dataset.read(2, window=window)
    vh_post_1=dataset.read(3, window=window)
    vv_post_1=dataset.read(4, window=window)
    
    
with rasterio.open(sar_image_path_2) as dataset:

    vh_pre_1=dataset.read(1, window=window)
    vv_pre_1=dataset.read(2, window=window)
    vh_post_2=dataset.read(4, window=window)
    vv_post_2=dataset.read(5, window=window)

    # meta=dataset.meta.copy()

epsilon = 1e-6
vh_pre_1=np.nan_to_num(vh_pre_1,nan=0)
vv_pre_1=np.nan_to_num(vv_pre_1,nan=0)
vh_pre_2=np.nan_to_num(vh_pre_2,nan=0)
vv_pre_2=np.nan_to_num(vv_pre_2,nan=0)
vh_post_1=np.nan_to_num(vh_post_1,nan=0)
vv_post_1=np.nan_to_num(vv_post_1,nan=0)
vh_post_2=np.nan_to_num(vh_post_2,nan=0)
vv_post_2=np.nan_to_num(vv_post_2,nan=0)

vh_pre_1=np.where(vh_pre_1 <=0,epsilon, vh_pre_1)
vv_pre_1=np.where(vv_pre_1 <=0,epsilon,vv_pre_1 )
vh_pre_2=np.where(vh_pre_2 <=0,epsilon, vh_pre_2)
vv_pre_2=np.where( vv_pre_2<=0,epsilon, vv_pre_2)
vh_post_1=np.where(vh_post_1 <=0,epsilon,vh_post_1 )
vv_post_1=np.where(vv_post_1 <=0,epsilon, vv_post_1)
vh_post_2=np.where(vh_post_2 <=0,epsilon, vh_post_2)
vv_post_2=np.where( vv_post_2<=0,epsilon, vv_post_2)

pre_1=(10*np.log10(vv_pre_1))-(10*np.log10(vh_pre_1))
pre_2=(10*np.log10(vv_pre_2))-(10*np.log10(vh_pre_2))
post_1=(10*np.log10(vv_post_1))-(10*np.log10(vh_post_1))
post_2=(10*np.log10(vv_post_2))-(10*np.log10(vh_post_2))

meta.update({"count":4})

with rasterio.open(output_image_path, 'w', **meta) as dest:
    dest.write(pre_1, 1)  
    dest.write(pre_2, 2)  
    dest.write(post_1, 3)  
    dest.write(post_2, 4) 




# save in order: VH_PRE, VV_PRE, VH_POST, VV_POST
# output_ordered_image_path="output/sar/1_eaton_s1_mul_arranged.tif"
# with rasterio.open(mul_sar_image_path) as dataset:
#     vh_pre_1=dataset.read(1)
#     vv_pre_1=dataset.read(2)
#     vh_post_2=dataset.read(3)
#     vv_post_2=dataset.read(4)
#     vh_pre_2=dataset.read(5)
#     vv_pre_2=dataset.read(6)
#     vh_post_1=dataset.read(7)
#     vv_post_1=dataset.read(8)
#     meta=dataset.meta.copy()

# # Write the clipped data to a new file
# with rasterio.open(output_ordered_image_path, 'w', **meta) as dest:
#     dest.write(vh_pre_1, 1)  
#     dest.write(vv_pre_1, 2)  
#     dest.write(vh_pre_2, 3)  
#     dest.write(vv_pre_2, 4) 
#     dest.write(vh_post_1, 5)  
#     dest.write(vv_post_1, 6)  
#     dest.write(vh_post_2, 7)  
#     dest.write(vv_post_2, 8) 

In [15]:
output_mul_sar_crop_path="output/sar/2_eaton_s1_mul_arranged_crop.tif"
with rasterio.open(output_ordered_image_path) as dataset:
        # Get the window for the specified extent
        window = from_bounds(*clip_extent, dataset.transform)
        # Read the data in that window
        # clipped_data = dataset.read(1, window=window)  # Read the first band in the window
        
        # Get the new transform for the clipped data
        clipped_transform = dataset.window_transform(window)
        clipped_data = dataset.read(1, window=window)
        # Define the metadata for the clipped raster
        clipped_meta = dataset.meta.copy()
        clipped_meta.update({
            'height': clipped_data.shape[0],
            'width': clipped_data.shape[1],
            'transform': clipped_transform
        })
        count=dataset.count

        # bands = dataset.read()
        
        # Write the clipped data to a new file
        with rasterio.open(output_mul_sar_crop_path, 'w', **clipped_meta) as dest:
            for i in range(count):
                clipped_data = dataset.read(i+1, window=window)
                dest.write(clipped_data, i+1)  # Write the first band

In [ ]:
# save in order: VH_PRE, VV_PRE, VH_POST, VV_POST
output_mul_sar_ready_path="output/sar/3_eaton_s1_mul_ready.tif"
epsilon = 1e-6
with rasterio.open(output_mul_sar_crop_path) as dataset:
    vh_pre_1=dataset.read(1)
    vv_pre_1=dataset.read(2)
    vh_pre_2=dataset.read(3)
    vv_pre_2=dataset.read(4)
    vh_post_1=dataset.read(5)
    vv_post_1=dataset.read(6)
    vh_post_2=dataset.read(7)
    vv_post_2=dataset.read(8)

    vh_pre_1=np.nan_to_num(vh_pre_1,nan=0)
    vv_pre_1=np.nan_to_num(vv_pre_1,nan=0)
    vh_pre_2=np.nan_to_num(vh_pre_2,nan=0)
    vv_pre_2=np.nan_to_num(vv_pre_2,nan=0)
    vh_post_1=np.nan_to_num(vh_post_1,nan=0)
    vv_post_1=np.nan_to_num(vv_post_1,nan=0)
    vh_post_2=np.nan_to_num(vh_post_2,nan=0)
    vv_post_2=np.nan_to_num(vv_post_2,nan=0)

    vh_pre_1=np.where(vh_pre_1 <=0,epsilon, vh_pre_1)
    vv_pre_1=np.where(vv_pre_1 <=0,epsilon,vv_pre_1 )
    vh_pre_2=np.where(vh_pre_2 <=0,epsilon, vh_pre_2)
    vv_pre_2=np.where( vv_pre_2<=0,epsilon, vv_pre_2)
    vh_post_1=np.where(vh_post_1 <=0,epsilon,vh_post_1 )
    vv_post_1=np.where(vv_post_1 <=0,epsilon, vv_post_1)
    vh_post_2=np.where(vh_post_2 <=0,epsilon, vh_post_2)
    vv_post_2=np.where( vv_post_2<=0,epsilon, vv_post_2)

    pre_1=(10*np.log10(vv_pre_1))-(10*np.log10(vh_pre_1))
    pre_2=(10*np.log10(vv_pre_2))-(10*np.log10(vh_pre_2))
    post_1=(10*np.log10(vv_post_1))-(10*np.log10(vh_post_1))
    post_2=(10*np.log10(vv_post_2))-(10*np.log10(vh_post_2))

    meta=dataset.meta.copy()
    meta.update({"count":4})


# Write the clipped data to a new file
with rasterio.open(output_mul_sar_ready_path, 'w', **meta) as dest:
    dest.write(pre_1, 1)  
    dest.write(pre_2, 2)  
    dest.write(post_1, 3)  
    dest.write(post_2, 4) 